# 모듈 데이터

In [1]:
!pip install -q joblib==1.3
import joblib
!pip install -q catboost
!pip install --upgrade -q xgboost

In [2]:
!pip install -q pycaret
from pycaret.regression import *

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
!pip install -q optuna

In [5]:
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

from sklearn.ensemble import VotingRegressor
from sklearn.metrics import mean_squared_error

In [6]:
from sklearn.metrics import mean_squared_log_error

In [7]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler

In [8]:
raw_train=pd.read_csv('/content/train_abalon.csv')
raw_test=pd.read_csv('/content/test_abalon.csv')

In [9]:
display(raw_train.info()),display(raw_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94792 entries, 0 to 94791
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Length          94792 non-null  float64
 1   Diameter        94792 non-null  float64
 2   Height          94792 non-null  float64
 3   Whole weight    94792 non-null  float64
 4   Whole weight.1  94792 non-null  float64
 5   Whole weight.2  94792 non-null  float64
 6   Shell weight    94792 non-null  float64
 7   Rings           94792 non-null  int64  
 8   d_l             94792 non-null  float64
 9   h_l             94792 non-null  float64
 10  sw_ww           94792 non-null  float64
 11  mw              94792 non-null  float64
 12  F               94792 non-null  int64  
 13  I               94792 non-null  int64  
 14  M               94792 non-null  int64  
dtypes: float64(11), int64(4)
memory usage: 10.8 MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60411 entries, 0 to 60410
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Sex             60411 non-null  object 
 1   Length          60411 non-null  float64
 2   Diameter        60411 non-null  float64
 3   Height          60411 non-null  float64
 4   Whole weight    60411 non-null  float64
 5   Whole weight.1  60411 non-null  float64
 6   Whole weight.2  60411 non-null  float64
 7   Shell weight    60411 non-null  float64
 8   d_l             60411 non-null  float64
 9   h_l             60411 non-null  float64
 10  sw_ww           60411 non-null  float64
 11  mw              60411 non-null  float64
 12  F               60411 non-null  int64  
 13  I               60411 non-null  int64  
 14  M               60411 non-null  int64  
dtypes: float64(11), int64(3), object(1)
memory usage: 6.9+ MB


None

(None, None)

In [10]:
raw_train_x=raw_train.drop('Rings',axis=1)
raw_train_y=raw_train['Rings']

In [11]:
display(raw_train_x.head()),display(raw_test.head()),raw_train_y.head(),raw_train_x.shape,raw_test.shape,raw_train_y.shape,display(raw_train_x.info()),display(raw_test.info())

,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,d_l,h_l,sw_ww,mw,F,I,M
0,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,0.781818,0.272727,0.311082,0.415500,1,0,0
1,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,0.777778,0.230159,0.283186,0.621500,1,0,0
2,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,0.687500,0.156250,0.238095,0.009833,0,1,0
3,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,0.798319,0.252101,0.273373,0.498500,0,0,1
4,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,0.765766,0.234234,0.252558,0.437167,0,1,0


,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,d_l,h_l,sw_ww,mw,F,I,M
0,M,0.645,0.475,0.155,1.2380,0.6185,0.3125,0.3005,0.736434,0.240310,0.242730,0.723000,0,0,1
1,M,0.580,0.460,0.160,0.9830,0.4785,0.2195,0.2750,0.793103,0.275862,0.279756,0.560333,0,0,1
2,M,0.560,0.420,0.140,0.8395,0.3525,0.1845,0.2405,0.750000,0.250000,0.286480,0.458833,0,0,1
3,M,0.570,0.490,0.145,0.8740,0.3525,0.1865,0.2350,0.859649,0.254386,0.268879,0.471000,0,0,1
4,I,0.415,0.325,0.110,0.3580,0.1575,0.0670,0.1050,0.783133,0.265060,0.293296,0.194167,0,1,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94792 entries, 0 to 94791
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Length          94792 non-null  float64
 1   Diameter        94792 non-null  float64
 2   Height          94792 non-null  float64
 3   Whole weight    94792 non-null  float64
 4   Whole weight.1  94792 non-null  float64
 5   Whole weight.2  94792 non-null  float64
 6   Shell weight    94792 non-null  float64
 7   d_l             94792 non-null  float64
 8   h_l             94792 non-null  float64
 9   sw_ww           94792 non-null  float64
 10  mw              94792 non-null  float64
 11  F               94792 non-null  int64  
 12  I               94792 non-null  int64  
 13  M               94792 non-null  int64  
dtypes: float64(11), int64(3)
memory usage: 10.1 MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60411 entries, 0 to 60410
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Sex             60411 non-null  object 
 1   Length          60411 non-null  float64
 2   Diameter        60411 non-null  float64
 3   Height          60411 non-null  float64
 4   Whole weight    60411 non-null  float64
 5   Whole weight.1  60411 non-null  float64
 6   Whole weight.2  60411 non-null  float64
 7   Shell weight    60411 non-null  float64
 8   d_l             60411 non-null  float64
 9   h_l             60411 non-null  float64
 10  sw_ww           60411 non-null  float64
 11  mw              60411 non-null  float64
 12  F               60411 non-null  int64  
 13  I               60411 non-null  int64  
 14  M               60411 non-null  int64  
dtypes: float64(11), int64(3), object(1)
memory usage: 6.9+ MB


None

(None,
 None,
 0    11
 1    11
 2     6
 3    10
 4     9
 Name: Rings, dtype: int64,
 (94792, 14),
 (60411, 15),
 (94792,),
 None,
 None)

In [12]:
#raw_test=raw_test.drop(columns=['id'],axis=1)

In [13]:
raw_test=raw_test.drop(columns=['Sex'],axis=1)
raw_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60411 entries, 0 to 60410
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Length          60411 non-null  float64
 1   Diameter        60411 non-null  float64
 2   Height          60411 non-null  float64
 3   Whole weight    60411 non-null  float64
 4   Whole weight.1  60411 non-null  float64
 5   Whole weight.2  60411 non-null  float64
 6   Shell weight    60411 non-null  float64
 7   d_l             60411 non-null  float64
 8   h_l             60411 non-null  float64
 9   sw_ww           60411 non-null  float64
 10  mw              60411 non-null  float64
 11  F               60411 non-null  int64  
 12  I               60411 non-null  int64  
 13  M               60411 non-null  int64  
dtypes: float64(11), int64(3)
memory usage: 6.5 MB


# normalize, log

In [14]:
sacler=MinMaxScaler()
raw_train_x_nom=sacler.fit_transform(raw_train_x)
raw_test_nom=sacler.transform(raw_test)

In [15]:
raw_train_y_log=np.log1p(raw_train_y)

In [16]:
raw_train_x_nom_df=pd.DataFrame(raw_train_x_nom,columns=raw_train_x.columns)
raw_test_nom_df=pd.DataFrame(raw_test_nom,columns=raw_test.columns)

In [17]:
display(raw_train_x_nom_df.info()),display(raw_test_nom_df.info()),raw_train_y_log

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94792 entries, 0 to 94791
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Length          94792 non-null  float64
 1   Diameter        94792 non-null  float64
 2   Height          94792 non-null  float64
 3   Whole weight    94792 non-null  float64
 4   Whole weight.1  94792 non-null  float64
 5   Whole weight.2  94792 non-null  float64
 6   Shell weight    94792 non-null  float64
 7   d_l             94792 non-null  float64
 8   h_l             94792 non-null  float64
 9   sw_ww           94792 non-null  float64
 10  mw              94792 non-null  float64
 11  F               94792 non-null  float64
 12  I               94792 non-null  float64
 13  M               94792 non-null  float64
dtypes: float64(14)
memory usage: 10.1 MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60411 entries, 0 to 60410
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Length          60411 non-null  float64
 1   Diameter        60411 non-null  float64
 2   Height          60411 non-null  float64
 3   Whole weight    60411 non-null  float64
 4   Whole weight.1  60411 non-null  float64
 5   Whole weight.2  60411 non-null  float64
 6   Shell weight    60411 non-null  float64
 7   d_l             60411 non-null  float64
 8   h_l             60411 non-null  float64
 9   sw_ww           60411 non-null  float64
 10  mw              60411 non-null  float64
 11  F               60411 non-null  float64
 12  I               60411 non-null  float64
 13  M               60411 non-null  float64
dtypes: float64(14)
memory usage: 6.5 MB


None

(None,
 None,
 0        2.484907
 1        2.484907
 2        1.945910
 3        2.397895
 4        2.302585
            ...   
 94787    2.484907
 94788    2.397895
 94789    2.302585
 94790    2.397895
 94791    2.564949
 Name: Rings, Length: 94792, dtype: float64)

# 옵튜나 튜닝
- 모든 데이터를 사용해 튜닝

In [22]:
from sklearn.model_selection import StratifiedKFold
from sklearn.compose import TransformedTargetRegressor
from xgboost import XGBRegressor
from sklearn.base import clone
from sklearn.metrics import mean_squared_log_error

### 기능설명




```
- 1. 목적함수
- 2. creat_study
- 3. sampler
- 4.study.optimize
```
1. 목적함수
- 파라미터
- 모델
- kfold : 데이터 분할 설정
- cross_cal_score : 분할된 데이터를 통한 교차검증
- 스코어링 및 출력

2. creat_study
- direction : 결과의 목표 방향설정, 양방향을 설정할경우 minimize로 변경됨
- sampler : 샘플링전략
- study_name(str: none) : 자동으로 새로운 스터디네임을 할당함
-load_if_exists : study_name 중복 처리에 관한 변수
  - false 일 경우 중복 study_name이 등장하면 error 발생
  - true 기존 study_name의 결과를 출력함

3. sampler
  - base sampler
    - 랜덤 샘플링
    - 각 샘플이 선택될 확률 설정 가능

  - grid sampler
    - 격자로 정의된 값들의 모든 조합을 탐색
  
  - random sampler
    - 랜덤 샘플링
    - 빠른 작업 속도

  - TPE sampler
    - 최초 랜덤 샘플링
    - 이전 결과를 활용한 샘플링
    - 효울적 탐색
    - 다변량 최적화 가능

4. study.optimize
- 목적함함수를 create_study에 맞게 최적화함
- sampler (def: TPE)
- n_trials, timeout으로 반복 제어




## catboost

In [ ]:
# 코드 참고
# 따로 fit할 필요 없음
# 평가지표 rmsle
def cb_objective(trial):
    # 탐색할 파라미터, 탐색 구간 설정
    params_cb = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 5000),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 1),
        "max_depth": trial.suggest_int("max_depth", 4, 12),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-8, 100),
        "random_strength": trial.suggest_loguniform("random_strength", 1e-8, 100),
        "bagging_temperature": trial.suggest_loguniform("bagging_temperature", 0.01, 100.00),
        "od_type": trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
        "metric_period": trial.suggest_categorical("metric_period", [10, 20, 30]),
        "random_state": 123
    }
    cb_model = CatBoostRegressor(**params_cb)
    cv = KFold(n_splits=7, shuffle=True, random_state=123)
    score_log = cross_val_score(cb_model, raw_train_x_nom_df, raw_train_y_log, cv=cv, n_jobs=-1,
                             scoring='neg_mean_squared_error')
    score_tmp1 = np.expm1(np.mean(-1*score_log))
    score = np.sqrt(score_tmp1)
    # mean은 cv 결과들 평균값 계산
    return score

study_cat = optuna.create_study(direction='minimize',
                                sampler=optuna.samplers.TPESampler(seed=123,
                                                                   n_startup_trials=20,
                                                                   multivariate=True),
                                study_name='optuna_catboost')
study_cat.optimize(cb_objective, n_trials=20)
best_params_cat = study_cat.best_params
best_params_cat

In [24]:
cb_alldata_tune_params={'n_estimators': 3513,
 'learning_rate': 0.03734897347801035,
 'max_depth': 6,
 'l2_leaf_reg': 0.003259475273606692,
 'random_strength': 0.15656320308475058,
 'bagging_temperature': 0.492522233779106,
 'od_type': 'IncToDec',
 'metric_period': 10}

## lgb

In [ ]:
def lgbm_objective(trial):
    # 모델 생성, 학습
    params_lgbm = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 5000),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 1),
        "max_depth": trial.suggest_int("max_depth", 4, 16),
        'num_leaves': trial.suggest_int("num_leaves", 20, 3000, step=10),
        'min_child_samples' : trial.suggest_int("min_child_samples", 100, 1000, step=10),
        'subsample': trial.suggest_float("subsample", 0.4, 1.0, step=0.1),
        'colsample_bytree': trial.suggest_float("colsample_bytree", 0.4, 1.0, step=0.1),
        "random_state": 123
    }
    lgbm_model = LGBMRegressor(**params_lgbm)
    cv = KFold(n_splits=7, shuffle=True, random_state=123)
    score_log = cross_val_score(lgbm_model, raw_train_x_nom_df, raw_train_y_log, cv=cv, n_jobs=-1,
                             scoring='neg_mean_squared_error')
    score_tmp1 = np.expm1(np.mean(-1*score_log))
    score = np.sqrt(score_tmp1)
    # mean은 cv 결과들 평균값 계산
    return score
study_lgbm = optuna.create_study(direction='minimize',
                                sampler=optuna.samplers.TPESampler(seed=123,
                                                                   n_startup_trials=20,
                                                                   multivariate=True),
                                study_name='optuna_lgbm')
study_lgbm.optimize(lgbm_objective, n_trials=20, n_jobs=-1)
best_params_lgbm = study_lgbm.best_params
best_params_lgbm


In [ ]:
lgb_alldata_tune_params={'n_estimators': 1421,
 'learning_rate': 0.012148128868574597,
 'max_depth': 15,
 'num_leaves': 870,
 'min_child_samples': 340,
 'subsample': 0.6000000000000001,
 'colsample_bytree': 0.5}

## xgb

In [ ]:
def xgb_objective(trial):
    # 모델 생성, 학습
    params_xgb = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 5000),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 1),
        "max_depth": trial.suggest_int("max_depth", 4, 12),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-8, 100),
        "random_strength": trial.suggest_loguniform("random_strength", 1e-8, 100),
        "bagging_temperature": trial.suggest_loguniform("bagging_temperature", 0.01, 100.00),
        "od_type": trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
        "metric_period": trial.suggest_categorical("metric_period", [10, 20, 30]),
        "random_state": 123
    }
    xgb_model = XGBRegressor(**params_xgb)
    cv = KFold(n_splits=7, shuffle=True, random_state=123)
    score_log = cross_val_score(xgb_model,  raw_train_x_nom_df, raw_train_y_log, cv=cv, n_jobs=-1,
                             scoring='neg_mean_squared_error')
    score_tmp1 = np.expm1(np.mean(-1*score_log))
    score = np.sqrt(score_tmp1)
    # mean은 cv 결과들 평균값 계산
    return score
study_xgb = optuna.create_study(direction='minimize',
                                sampler=optuna.samplers.TPESampler(seed=123,
                                                                   n_startup_trials=20,
                                                                   multivariate=True),
                                study_name='optuna_xgb')
study_xgb.optimize(xgb_objective, n_trials=20, n_jobs=-1)
best_params_xgb = study_xgb.best_params
best_params_xgb


In [ ]:
xgb_alldata_tune_params={'n_estimators': 505,
 'learning_rate': 0.049230611117457294,
 'max_depth': 6,
 'l2_leaf_reg': 0.005408016691095872,
 'random_strength': 3.828125968522393,
 'bagging_temperature': 0.012362661994054768,
 'od_type': 'Iter',
 'metric_period': 20}

# oof를 이용한 학습 및 예측
- 전체 데이터를 학습에 이용

## 함수생성

In [26]:
from sklearn.model_selection import StratifiedKFold
from sklearn.compose import TransformedTargetRegressor
from xgboost import XGBRegressor
from sklearn.base import clone
from sklearn.metrics import mean_squared_log_error

In [28]:
SEED = 42
N_SPLITS = 10
N_REPEATS = 5

- train
    - StratifiedKFold 교차 검증을 통해 훈련
    - 각 폴드에서 모델을 훈련하고 예측하여 예측값의 평균 생성
    - 폴드 별로 점수 출력
        - 전제 폴드에 대한 평가점수 반환

In [29]:
# 훈련을 위한 train 함수 생성
def train(model, X, y, n_splits=N_SPLITS, n_repeats=N_REPEATS):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)

    # 예측 결과를 저장 / 각각의 평가 점수 저장
    oof_preds = np.zeros(len(X), dtype=float)
    scores = []

    # 각각의 폴드별로 스플릿을 진행
    for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        y_pred = np.zeros_like(y_val, dtype=float)

        for i in range(n_repeats):
            m = clone(model)
            if n_repeats > 1:
                m.regressor.set_params(random_state=i)
            m.fit(X_train, y_train)
            y_pred += m.predict(X_val)

        y_pred /= n_repeats
        y_pred = y_pred.clip(1, 29)

        score = mean_squared_log_error(y_val, y_pred, squared=False)
        print(f'Fold {fold_idx + 1} Score: {score:.6f}')
        scores.append(score)

        oof_preds[val_idx] = y_pred

    final_score = np.array(scores).mean()
    print(f'\nFinal Score: {final_score:.6f}')

    return final_score, oof_preds

- predict
    - 주어진 모델을 활용하여 테스트의 예측 생성
    - 예측을 반복하고 평균값을 통해 최종 예측값 반환

In [30]:
def predict(model, X, y, X_test, n_repeats=N_REPEATS):
    y_pred = np.zeros(len(X_test), dtype=float)
    for i in range(n_repeats):
        m = clone(model)
        if n_repeats > 1:
            m.regressor.set_params(random_state=i)
        m.fit(X, y)
        y_pred += m.predict(X_test)
    y_pred /= n_repeats
    y_pred = y_pred.clip(1, 29)
    return y_pred

## catboost
- 0.14686

In [ ]:
cb_model = TransformedTargetRegressor(
    regressor=CatBoostRegressor(**cb_alldata_tune_params),
    func=np.log1p,
    inverse_func=np.expm1
)

scores, oof_preds = train(cb_model, raw_train_x_nom_df, raw_train_y)

In [ ]:
predictions = predict(cb_model, raw_train_x_nom_df, raw_train_y, raw_test_nom_df)

In [ ]:
#sample_submission['Rings'] = predictions
#sample_submission.to_csv('cb_no_outlier_fulldatatune_nom_log_optuma_oof_nosplit_3.csv', index=False)
#sample_submission.head()

## lgb
-  0.14763

In [ ]:
lgb_model = TransformedTargetRegressor(
    regressor=LGBMRegressor(**lgb_alldata_tune_params),
    func=np.log1p,
    inverse_func=np.expm1
)

scores, oof_preds = train(lgb_model, raw_train_x_nom_df, raw_train_y)

In [ ]:
predictions = predict(lgb_model, raw_train_x_nom_df, raw_train_y, raw_test_nom_df)

In [ ]:
#sample_submission['Rings'] = predictions
#sample_submission.to_csv('lgbm_no_outlier_nom_log_optuma_oof_nosplit_2.csv', index=False)
#sample_submission.head()

## xgb
- 0.14791

In [ ]:

xgb_model = TransformedTargetRegressor(
    regressor=XGBRegressor(**xgb_alldata_tune_params),
    func=np.log1p,
    inverse_func=np.expm1
)

scores, oof_preds = train(xgb_model, raw_train_x_nom_df, raw_train_y)

In [ ]:
predictions = predict(xgb_model, raw_train_x_nom_df, raw_train_y, raw_test_nom_df)

In [ ]:
#sample_submission['Rings'] = predictions
#sample_submission.to_csv('xgb_no_outlier_nom_log_optuma_oof_nosplit_2.csv', index=False)
#sample_submission.head()

- 외에도 oof 방식으로 voting과 stack(메타모델 lr)을 시도했으나 각각 0.14742, 0.14739로 catboost 단독보다 점수가 낮았음

## cb (비율 피처 제거)

- 0.14629
- catboost는 트리구조이기 때문에 normalize 유무가 상관없음

In [18]:
raw_train_x_nom_df = raw_train_x_nom_df.drop('d_l', axis=1)
raw_train_x_nom_df = raw_train_x_nom_df.drop('h_l', axis=1)
raw_train_x_nom_df = raw_train_x_nom_df.drop('sw_ww', axis=1)
raw_train_x_nom_df = raw_train_x_nom_df.drop('mw', axis=1)
raw_train_x_nom_df.head(1)

,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,F,I,M
0,0.641892,0.630252,0.132743,0.272534,0.220242,0.192232,0.237668,1.0,0.0,0.0


In [20]:
raw_test_nom_df = raw_test_nom_df.drop('d_l', axis=1)
raw_test_nom_df = raw_test_nom_df.drop('h_l', axis=1)
raw_test_nom_df = raw_test_nom_df.drop('sw_ww', axis=1)
raw_test_nom_df = raw_test_nom_df.drop('mw', axis=1)
raw_test_nom_df.head(1)

,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,F,I,M
0,0.77027,0.705882,0.137168,0.437755,0.415266,0.410797,0.297957,0.0,0.0,1.0


In [ ]:
cb_model = TransformedTargetRegressor(
    regressor=CatBoostRegressor(**cb_alldata_tune_params),
    func=np.log1p,
    inverse_func=np.expm1
)

scores, oof_preds = train(cb_model, raw_train_x_nom_df, raw_train_y)

In [ ]:
predictions = predict(cb_model, raw_train_x_nom_df, raw_train_y, raw_test_nom_df)

In [ ]:
#sample_submission['Rings'] = predictions
#sample_submission.to_csv('xgb_no_outlier_nom_log_optuma_oof_nosplit_.csv', index=False)
#sample_submission.head()